# CICIDS2017

## Data preparation

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import zipfile
import os
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import time
import glob
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

In [ ]:
!pip install kaggle
!mkdir ~/.kaggle
!cp /content/kaggle.json /root/.kaggle/
!chmod 600 /root/.kaggle/kaggle.json
!kaggle datasets download -d cicdataset/cicids2017

mkdir: cannot create directory ‘/root/.kaggle’: File exists
cicids2017.zip: Skipping, found more recently modified local copy (use --force to force download)


In [ ]:
# Open the zip file
with zipfile.ZipFile("/content/cicids2017.zip", "r") as zip_ref:
    # Extract all contents of the zip file to the current working directory
    zip_ref.extractall()

In [ ]:
file_pattern = "/content/MachineLearningCSV/MachineLearningCVE/*.csv"
csv_files = glob.glob(file_pattern)

combined_df = pd.DataFrame()

for csv_file in csv_files:
    df = pd.read_csv(csv_file)
    combined_df = combined_df.append(df, ignore_index=True)

<ipython-input-16-c6df76b0bcb5>:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  combined_df = combined_df.append(df, ignore_index=True)
<ipython-input-16-c6df76b0bcb5>:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  combined_df = combined_df.append(df, ignore_index=True)
<ipython-input-16-c6df76b0bcb5>:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  combined_df = combined_df.append(df, ignore_index=True)
<ipython-input-16-c6df76b0bcb5>:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  combined_df = combined_df.append(df, ignore_index=True)
<ipython-input-16-c6df76b0bcb5>:8: FutureWarning: The frame.append method is deprecated and will

In [ ]:
combined_df[" Label"].unique()

array(['BENIGN', 'DoS slowloris', 'DoS Slowhttptest', 'DoS Hulk',
       'DoS GoldenEye', 'Heartbleed', 'Infiltration', 'DDoS',
       'FTP-Patator', 'SSH-Patator', 'PortScan', 'Bot',
       'Web Attack � Brute Force', 'Web Attack � XSS',
       'Web Attack � Sql Injection'], dtype=object)

In [ ]:
(combined_df[" Label"] == "BENIGN").sum()

2273097

In [ ]:
selected_normal_samples = combined_df[combined_df[" Label"] == "BENIGN"].sample(n=557646, random_state=42)

In [ ]:
dataset = selected_normal_samples.append(combined_df[combined_df[" Label"] != "BENIGN"])

<ipython-input-19-44e1ed068abc>:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dataset = selected_normal_samples.append(combined_df[combined_df[" Label"] != "BENIGN"])


In [ ]:
dataset.describe()

,Destination Port,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,...,act_data_pkt_fwd,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min
count,1.115292e+06,1.115292e+06,1.115292e+06,1.115292e+06,1.115292e+06,1.115292e+06,1.115292e+06,1.115292e+06,1.115292e+06,1.115292e+06,...,1.115292e+06,1.115292e+06,1.115292e+06,1.115292e+06,1.115292e+06,1.115292e+06,1.115292e+06,1.115292e+06,1.115292e+06,1.115292e+06
mean,6.000525e+03,2.027992e+07,7.334779e+00,7.624194e+00,4.275507e+02,1.196020e+04,1.725663e+02,1.248014e+01,4.561581e+01,6.020730e+01,...,4.205889e+00,-1.943999e+03,8.934331e+04,3.275669e+04,1.387994e+05,6.967779e+04,1.535852e+07,8.776434e+05,1.602671e+07,1.470877e+07
std,1.553054e+04,3.780332e+07,5.838138e+02,7.666395e+02,1.390076e+04,1.777430e+06,5.840563e+02,5.552887e+01,1.533200e+02,2.269747e+02,...,5.438679e+02,9.052437e+05,6.526687e+05,3.738567e+05,9.372091e+05,5.914127e+05,3.250418e+07,6.317248e+06,3.336306e+07,3.228805e+07
min,0.000000e+00,-2.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,-5.368707e+08,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,8.000000e+01,7.400000e+01,1.000000e+00,1.000000e+00,2.000000e+00,0.000000e+00,2.000000e+00,0.000000e+00,2.000000e+00,0.000000e+00,...,0.000000e+00,2.000000e+01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
50%,8.000000e+01,4.668300e+04,2.000000e+00,2.000000e+00,4.700000e+01,1.110000e+02,3.000000e+01,0.000000e+00,1.850000e+01,0.000000e+00,...,1.000000e+00,2.400000e+01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
75%,4.430000e+02,8.731766e+06,6.000000e+00,5.000000e+00,3.140000e+02,5.926000e+03,2.170000e+02,6.000000e+00,4.800000e+01,7.781181e+01,...,2.000000e+00,3.200000e+01,4.000000e+00,0.000000e+00,4.000000e+00,4.000000e+00,6.025854e+06,0.000000e+00,6.026833e+06,5.596163e+06
max,6.553300e+04,1.200000e+08,2.186580e+05,2.912600e+05,1.290000e+07,6.410014e+08,2.482000e+04,2.325000e+03,5.940857e+03,7.125597e+03,...,2.075010e+05,9.300000e+01,1.070000e+08,7.000000e+07,1.070000e+08,1.070000e+08,1.200000e+08,7.690000e+07,1.200000e+08,1.200000e+08


In [ ]:
feature_dict = {f"feat{i}": col_name for i, col_name in enumerate(dataset.columns, start=1)}

In [ ]:
feature_dict

In [ ]:
dataset.head(3)

In [ ]:
dataset.iloc[:, -1] = dataset.iloc[:, -1].apply(lambda x: 0 if x == "BENIGN" else 1)

<ipython-input-21-de8ef5e7dce8>:1: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  dataset.iloc[:, -1] = dataset.iloc[:, -1].apply(lambda x: 0 if x == "BENIGN" else 1)


In [ ]:
dataset.tail(3)

In [ ]:
dataset.columns = list(feature_dict.keys())
dataset.head(3)

,feat1,feat2,feat3,feat4,feat5,feat6,feat7,feat8,feat9,feat10,...,feat70,feat71,feat72,feat73,feat74,feat75,feat76,feat77,feat78,feat79
1377005,5510,47,2,0,4,0,2,2,2.000,0.000000,...,24,0.00000,0.00000,0,0,0.000,0.000000,0,0,0
2755115,57170,53,1,2,0,0,0,0,0.000,0.000000,...,32,0.00000,0.00000,0,0,0.000,0.000000,0,0,0
1494705,80,116029368,16,14,582,712,498,0,36.375,123.109098,...,20,48811.63636,20680.73497,111165,42387,9999286.091,6696.651215,10000000,9980071,0


In [ ]:
ind = []
for i in range(1, 80):
  ind.append(len(dataset["feat" + str(i)].unique()))
print(ind)

[36339, 527333, 777, 955, 9983, 27224, 3850, 257, 40946, 82760, 4077, 458, 53413, 75143, 655210, 581746, 553238, 394831, 280247, 56093, 193138, 266753, 248222, 185359, 43982, 215301, 295378, 331709, 206645, 24178, 2, 1, 2, 1, 1941, 2101, 574283, 501800, 187, 4636, 79719, 161673, 158629, 2, 2, 2, 2, 2, 2, 2, 2, 18, 77879, 40946, 53412, 1941, 1, 1, 1, 1, 1, 1, 777, 9983, 955, 27228, 6925, 7229, 590, 23, 106244, 51736, 103505, 75070, 77314, 49387, 61668, 90417, 2]


In [ ]:
ind = []
for i in range(1, 80):
  if len(dataset["feat" + str(i)].unique()) == 1:
    ind.append("feat" + str(i))

for key in ind:
    values = feature_dict[key]
    print(f"{key}: {values}")

feat32:  Bwd PSH Flags
feat34:  Bwd URG Flags
feat57: Fwd Avg Bytes/Bulk
feat58:  Fwd Avg Packets/Bulk
feat59:  Fwd Avg Bulk Rate
feat60:  Bwd Avg Bytes/Bulk
feat61:  Bwd Avg Packets/Bulk
feat62: Bwd Avg Bulk Rate


In [ ]:
dataset["feat34"]

1377005    0
2755115    0
1494705    0
1519480    0
827122     0
          ..
2753262    0
2753361    0
2753610    0
2753619    0
2753663    0
Name: feat34, Length: 1115292, dtype: int64

In [ ]:
for item in ind: dataset.drop(item, axis=1, inplace=True)

In [ ]:
column_types = dataset.dtypes

for column_name, column_type in column_types.items():
    print(f"Column '{column_name}' has type: {column_type}")

In [ ]:
dataset.tail(3)

,feat1,feat2,feat3,feat4,feat5,feat6,feat7,feat8,feat9,feat10,...,feat70,feat71,feat72,feat73,feat74,feat75,feat76,feat77,feat78,feat79
2753610,80,5009163,4,4,599,2021,599,0,149.75,299.5,...,32,0.0,0.0,0,0,0.0,0.0,0,0,1
2753619,80,73,1,1,0,0,0,0,0.00,0.0,...,32,0.0,0.0,0,0,0.0,0.0,0,0,1
2753663,80,109,1,1,0,0,0,0,0.00,0.0,...,32,0.0,0.0,0,0,0.0,0.0,0,0,1


In [ ]:
dataset = dataset[~dataset.isin([np.nan, np.inf, -np.inf]).any(1)]

train_data, test_data = train_test_split(dataset, test_size=0.2, random_state=42)

train_data = train_data.reset_index(drop=True)
test_data = test_data.reset_index(drop=True)

train_label = train_data["feat79"]
test_label = test_data["feat79"]

scaler = StandardScaler()
train_data = scaler.fit_transform(train_data.drop(columns=['feat79']))
test_data = scaler.transform(test_data.drop(columns=['feat79']))

train_data = pd.DataFrame(train_data)
test_data = pd.DataFrame(test_data)

train_data['label'] = train_label
test_data['label'] = test_label

train_data.columns = dataset.columns
test_data.columns = dataset.columns

train_data.to_csv('/content/drive/MyDrive/cicids/output_cicids_train_file.csv', index=False)
test_data.to_csv('/content/drive/MyDrive/cicids/output_cicids_test_file.csv', index=False)

<ipython-input-34-08dd59c7895b>:1: FutureWarning: In a future version of pandas all arguments of DataFrame.any and Series.any will be keyword-only.
  dataset = dataset[~dataset.isin([np.nan, np.inf, -np.inf]).any(1)]


In [ ]:
train_data.tail(3)

,feat1,feat2,feat3,feat4,feat5,feat6,feat7,feat8,feat9,feat10,...,feat70,feat71,feat72,feat73,feat74,feat75,feat76,feat77,feat78,feat79
891023,-0.381121,2.525640,0.012551,0.006722,0.028554,-0.006485,1.057211,-0.224664,0.058674,0.599217,...,0.002015,0.017831,0.285212,0.404891,-0.017447,-0.165246,-0.137458,-0.181079,-0.147105,0
891024,-0.381121,-0.536867,-0.008912,-0.009623,-0.028381,-0.006759,-0.295458,-0.224664,-0.297240,-0.265415,...,0.002029,-0.137759,-0.089031,-0.149450,-0.118357,-0.472836,-0.139175,-0.480731,-0.455817,1
891025,-0.357737,-0.536866,-0.010445,-0.008455,-0.027991,-0.006756,-0.285210,-0.116701,-0.258236,-0.265415,...,0.002015,-0.137759,-0.089031,-0.149450,-0.118357,-0.472836,-0.139175,-0.480731,-0.455817,0


## LSTM model

In [ ]:
input_shape = (1, 70)

model = Sequential()
model.add(LSTM(64, input_shape=input_shape))
model.add(Dense(1))

model.compile(optimizer='adam', loss='mean_squared_error', metrics=['accuracy'])

In [ ]:
batch_size = 10000
epochs = 10
file_path = '/content/drive/MyDrive/cicids/output_cicids_train_file.csv'

for i, chunk in enumerate(pd.read_csv(file_path, chunksize=batch_size)):
    X_train = chunk.drop('feat79', axis=1)
    y_train = chunk['feat79']

    sequence_length = 1
    X_train = np.array([X_train[i:i+sequence_length] for i in range(len(X_train) - sequence_length + 1)])
    y_train = y_train[sequence_length - 1:]

    # Train the model on the current chunk
    model.fit(X_train, y_train, epochs=epochs, verbose=0)
    print("end chunk ", i)


"""
# Evaluate the model on the entire dataset or a test set if available
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print(f'Test Loss: {test_loss}, Test Accuracy: {test_accuracy}')"""

end chunk  0
end chunk  1
end chunk  2
end chunk  3
end chunk  4
end chunk  5
end chunk  6
end chunk  7
end chunk  8
end chunk  9
end chunk  10
end chunk  11
end chunk  12
end chunk  13
end chunk  14
end chunk  15
end chunk  16
end chunk  17
end chunk  18
end chunk  19
end chunk  20
end chunk  21
end chunk  22
end chunk  23
end chunk  24
end chunk  25
end chunk  26
end chunk  27
end chunk  28
end chunk  29
end chunk  30
end chunk  31
end chunk  32
end chunk  33
end chunk  34
end chunk  35
end chunk  36
end chunk  37
end chunk  38
end chunk  39
end chunk  40
end chunk  41
end chunk  42
end chunk  43
end chunk  44
end chunk  45
end chunk  46
end chunk  47
end chunk  48
end chunk  49
end chunk  50
end chunk  51
end chunk  52
end chunk  53
end chunk  54
end chunk  55
end chunk  56
end chunk  57
end chunk  58
end chunk  59
end chunk  60
end chunk  61
end chunk  62
end chunk  63
end chunk  64
end chunk  65
end chunk  66
end chunk  67
end chunk  68
end chunk  69
end chunk  70
end chunk  71
en

"\n# Evaluate the model on the entire dataset or a test set if available\ntest_loss, test_accuracy = model.evaluate(X_test, y_test)\nprint(f'Test Loss: {test_loss}, Test Accuracy: {test_accuracy}')"

In [ ]:
model_architecture = model.to_json()

with open('lstm_model.json', 'w') as json_file:
    json_file.write(model_architecture)


model.save_weights('lstm_model.h5')

In [ ]:
from tensorflow import keras

with open('/content/lstm_model.json', 'r') as json_file:
    loaded_model_json = json_file.read()
model = tf.keras.models.model_from_json(loaded_model_json)

model.load_weights('/content/lstm_model.h5')
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['accuracy'])

model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_2 (LSTM)               (None, 64)                34560     
                                                                 
 dense_2 (Dense)             (None, 1)                 65        
                                                                 
Total params: 34625 (135.25 KB)
Trainable params: 34625 (135.25 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
test_set = pd.read_csv("/content/drive/MyDrive/cicids/output_cicids_test_file.csv")

In [ ]:
X_test = test_set.drop('feat79', axis=1)
y_test = test_set['feat79']

sequence_length = 5
X_test = np.array([X_test[i:i+sequence_length] for i in range(len(X_test) - sequence_length + 1)])
y_test = y_test[sequence_length - 1:]

In [ ]:
score = model.evaluate(X_test, y_test, verbose = 0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 0.018710732460021973
Test accuracy: 0.9789811968803406


In [ ]:
predictions = model.predict(X_test)

6962/6962 [==============================] - 20s 3ms/step


In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score

predictions = (predictions > 0.6).astype(np.int32)

# Calculate precision
precision = precision_score(y_test, predictions)

# Calculate recall
recall = recall_score(y_test, predictions)

# Calculate F1 score
f1 = f1_score(y_test, predictions)


print("Precision: {:.4f}".format(precision))
print("Recall: {:.4f}".format(recall))
print("F1 Score: {:.4f}".format(f1))

Precision: 0.9694
Recall: 0.9888
F1 Score: 0.9790


## Naive bayes model

In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

In [ ]:
naive_bayes_classifier = GaussianNB()

In [ ]:
batch_size = 10000
file_path = '/content/drive/MyDrive/cicids/output_cicids_train_file.csv'

for i, chunk in enumerate(pd.read_csv(file_path, chunksize=batch_size)):
    X_train = chunk.drop('feat79', axis=1)
    y_train = chunk['feat79']

    # Train the model on the current chunk
    naive_bayes_classifier.fit(X_train, y_train)
    print("end chunk ", i)

end chunk  0
end chunk  1
end chunk  2
end chunk  3
end chunk  4
end chunk  5
end chunk  6
end chunk  7
end chunk  8
end chunk  9
end chunk  10
end chunk  11
end chunk  12
end chunk  13
end chunk  14
end chunk  15
end chunk  16
end chunk  17
end chunk  18
end chunk  19
end chunk  20
end chunk  21
end chunk  22
end chunk  23
end chunk  24
end chunk  25
end chunk  26
end chunk  27
end chunk  28
end chunk  29
end chunk  30
end chunk  31
end chunk  32
end chunk  33
end chunk  34
end chunk  35
end chunk  36
end chunk  37
end chunk  38
end chunk  39
end chunk  40
end chunk  41
end chunk  42
end chunk  43
end chunk  44
end chunk  45
end chunk  46
end chunk  47
end chunk  48
end chunk  49
end chunk  50
end chunk  51
end chunk  52
end chunk  53
end chunk  54
end chunk  55
end chunk  56
end chunk  57
end chunk  58
end chunk  59
end chunk  60
end chunk  61
end chunk  62
end chunk  63
end chunk  64
end chunk  65
end chunk  66
end chunk  67
end chunk  68
end chunk  69
end chunk  70
end chunk  71
en

In [ ]:
test_set = pd.read_csv("/content/drive/MyDrive/cicids/output_cicids_test_file.csv")

X_test = test_set.drop('feat79', axis=1)
y_test = test_set['feat79']

In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score

predictions = naive_bayes_classifier.predict(X_test)

predictions = (predictions > 0.6).astype(np.int32)

# Calculate precision
precision = precision_score(y_test, predictions)

# Calculate recall
recall = recall_score(y_test, predictions)

# Calculate F1 score
f1 = f1_score(y_test, predictions)


print("Precision: {:.4f}".format(precision))
print("Recall: {:.4f}".format(recall))
print("F1 Score: {:.4f}".format(f1))

Precision: 0.8625
Recall: 0.8840
F1 Score: 0.8731


In [ ]:
# Confusion Matrix
conf_matrix = confusion_matrix(y_test, predictions)
print("Confusion Matrix:\n", conf_matrix)

Confusion Matrix:
 [[95464 15724]
 [12942 98627]]
